In [1]:
import yaml
import data
from data.data_split import kfsplit
import numpy as np

In [2]:
import pandas as pd 
import os
import torch

from torch.cuda.amp import autocast as autocast, GradScaler
from sklearn.metrics import precision_score,recall_score,f1_score
from sklearn.model_selection import train_test_split
from torch.utils.data.distributed import DistributedSampler
import gc 
gc.enable()
from data.datasets import prepare_loaders

import wandb
from model.head import OriginModel,LR,get_model

import torch
import os
import gc

from model.trainer import train,predict
from utils.wandb_init import get_run,wandb_utils
from utils.train import get_score,train_set
from sklearn.metrics import precision_score,recall_score,f1_score
from data.read_data import select_data,select_data,TrainData,file_split,DataByWord
from data.data_split import kfsplit
from data.embedding_init import get_tokenizer,get_embedding
from data.datasets import DatasetRetriever
import nltk
from collections import defaultdict
from torchtext.data.utils import ngrams_iterator
import json
from tqdm import tqdm

2022-11-04 23:33:13.018534: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [3]:
yaml_path = "./config/train.yaml"
CONFIG,Logger,DEVICE = train_set(yaml_path,experimentName = None,upload = False,filename = "./test/logs",is_notebook = True)

CONFIG["run_db"] = False
# CONFIG["num_hidden_layers"] = 12
CONFIG["DATA_NUM_RATE"] = 0.2
# CONFIG["EPOCHS"] = 5
CONFIG["DEBUG_MODEL"] = False


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: adreambear. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [4]:
with open(os.path.join(CONFIG["INPUT_DIR"],"Schema"),"r") as f:
    line=f.readlines()[0].strip()
    column_name=line.split("\t")
    CONFIG["Logger"].info(f"column_name :{column_name}")
    CONFIG["COLUMN_NAME"]=column_name
    
if CONFIG["HARD_NEG"]:
    filter_rate,hard_rate=CONFIG["FILTER_RATE"],CONFIG["HARD_RATE"]
    CONFIG["Logger"].info("-"*10+f"filter_rate: {filter_rate} hard_rate: {hard_rate}"+"-"*10)
    trainDataObj=DataByWord(CONFIG,"train",column_name=column_name,filter_rate=filter_rate,hard_rate=hard_rate)
else:
    trainDataObj=DataByWord(CONFIG,"train",column_name=column_name)


trainData=trainDataObj.get_alldata()
# trainData=trainDataObj.get_data(select_word="coffee")

CONFIG["Logger"].info("-"*10+"trainData label's value_counts"+"-"*10)
CONFIG["Logger"].info(trainData["label"].value_counts())
CONFIG["Logger"].info("-"*10+"trainData fold's value_counts"+"-"*10)
CONFIG["Logger"].info(trainData["fold"].value_counts())

column_name :['Query', 'keyword', 'query_len', 'emotion_value', 'label']
----------trainData label's value_counts----------
0    190964
1    174230
Name: label, dtype: int64
----------trainData fold's value_counts----------
1.0    263965
0.0    101229
Name: fold, dtype: int64


In [5]:
trainDataObj.binary_word_list

['candy',
 'beans',
 'go',
 'superman',
 'coffee',
 'phoenix',
 'tabs',
 'live',
 'bars',
 'diesel',
 'truck drivers',
 'food',
 'fire',
 'girl',
 'white',
 'wet',
 'zip',
 'junk',
 'leaf',
 'shoes',
 'flour',
 'snowflake',
 'green',
 'space',
 'trees',
 'paint',
 'lemonade',
 'work',
 'k',
 'purple',
 'poison',
 'black',
 'blue',
 'salt',
 'jet',
 'ace',
 'buttons',
 'whites',
 'him',
 'bananas',
 'blues',
 'rope',
 'mother',
 'scratch',
 'mph',
 'water',
 'speed']

In [6]:
# from data.read_data import read_data_file

# i = trainDataObj.binary_word_list.index(select_word)
# file_path = os.path.join(CONFIG["INPUT_DIR"],trainDataObj.binary_word_list[i])
# positive_file = os.path.join(file_path,"Positive"+trainDataObj.data_type.capitalize())
# negative_file = os.path.join(file_path,"Negative"+trainDataObj.data_type.capitalize())
# positive_df = read_data_file(positive_file,\
#     CONFIG["DEBUG_MODEL"],data_num=CONFIG["DATA_NUM"],head=trainDataObj.column_name)

# negative_df = read_data_file(negative_file,\
#     CONFIG["DEBUG_MODEL"],data_num=CONFIG["DATA_NUM"],head=trainDataObj.column_name)

In [6]:
data_df=trainData
fold=0
run=None

In [7]:
for fold in range(CONFIG["NUM_FOLDS"]):

    if CONFIG["cross_val"]==False and fold>=1:
        break
    CONFIG["Logger"].info(f"----------fold {fold}----------")
    run=get_run(CONFIG,"Train",fold)
    model=get_model(CONFIG["model_name"])(CONFIG)
    # model=OriginModel(CONFIG)
    train(model,data_df,fold,CONFIG,DEVICE,run=run,trainDataObj=trainDataObj)
    
    # _,_,acc_train_lastepoch=predict(data_df,DEVICE,CONFIG,model_path=None,model=model)
    
    # CONFIG["Logger"].info(f"train{fold} acc in last epoch:{acc_train_lastepoch}")
    del model
    torch.cuda.empty_cache()
    if run!=None:
        run.finish()

gc.collect()

----------fold 0----------


get model : bert-base-uncased


----------train start----------
epoch 0 learning rate start  :0.0002


Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


  0%|          | 2/516 [00:03<12:34,  1.47s/it]

Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0


100%|██████████| 99/99 [00:47<00:00,  2.10it/s]  
EPOCH:1/10 train_loss:0.2672803997993469,val_loss:0.295006536173098
f1:0.9010195158937888,recall:0.9009671141668889,precesion:0.9026284338285702,val_acc:0.9009671141668889
one_epoch_time:4.471045692761739 min
best_model saved ,f1:0.9010195158937888,recall:0.9009671141668889,precesion:0.9026284338285702,val acc:0.9009671141668889
epoch 1 learning rate start  :0.00018
100%|██████████| 99/99 [00:46<00:00,  2.14it/s]  
EPOCH:2/10 train_loss:0.17958778301759284,val_loss:0.4279932641621792
f1:0.8984145897782222,recall:0.8983591658516829,precesion:0.8991622359553456,val_acc:0.8983591658516829
one_epoch_time:4.44686484336853 min
epoch 2 learning rate start  :0.00016
 22%|██▏       | 114/516 [00:49<02:50,  2.36it/s]